In [1]:
# wine data 불러오기 
import pandas as pd
dat_wine=pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/'
                     'wine/wine.data',header=None) # wine data 불러오기

from sklearn.model_selection import train_test_split
X,y=dat_wine.iloc[:,1:].values, dat_wine.iloc[:,0].values
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3, random_state=1,stratify=y)

## Logistic

: 표준화는 하지 않았음

In [23]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression(random_state=1)

# 초모수 결정
# param_range = [ 0.05, 0.1, 0.15]
param_range = [0.01, 0.03, 0.05]
param_grid = {'C': param_range,'multi_class' : ['ovr'], 'max_iter' : [1000, 2000, 5000]}
             
gs = GridSearchCV(estimator=lr, param_grid=param_grid,
                  scoring='accuracy', cv=10)
gs = gs.fit(X_train, y_train) 
print(gs.best_score_) # cv에서의 최고 accuracy (일부 성능)
print(gs.best_params_) # 초모수 결정 0.1 중심으로 다시해보기 0.05 중심으로 다시

# 최종 : training data를 다 쓴 것
clf = gs.best_estimator_
clf.fit(X_train, y_train)
print(clf.score(X_train,y_train)) 
print(clf.score(X_test, y_test)) # true 최종 성능

0.9602564102564102
{'C': 0.03, 'max_iter': 1000, 'multi_class': 'ovr'}
0.9596774193548387
0.8888888888888888


In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
gs = GridSearchCV(estimator=lr, param_grid=param_grid,
                  scoring='accuracy', cv=3)
scores = cross_val_score(gs, X, y, scoring='accuracy', cv=5)
print('CV accuracy: %.8f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.95539683 +/- 0.028


In [10]:
# 층화추출한 것
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold # 분할 내 범주비율이 전체데이터 범주비율과 유사하게
inner_cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=0) # shuffle 해 잘 섞음
outer_cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

gs = GridSearchCV(estimator=lr,
                  param_grid=param_grid,
                  scoring='accuracy', cv=inner_cv)
scores = cross_val_score(gs, X, y, scoring='accuracy', cv=outer_cv)

print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.950 +/- 0.020


## SVM

파이프라인 사용, 표준화 시킴

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
pipe_svc = make_pipeline(StandardScaler(), SVC(random_state=1))

# param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_range = [0.05, 0.1, 0.15, 0.2]
param_grid = [{'svc__C': param_range, 'svc__kernel': ['linear']}, 
              {'svc__C': param_range, 'svc__gamma': param_range,
               'svc__kernel': ['rbf', 'poly', 'sigmoid']}]
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid,
                  scoring='accuracy', cv=10)
gs = gs.fit(X_train, y_train)
print(gs.best_score_) # 1.0 중심의 c, 또는 gamma 까지 해서 다시해보기
print(gs.best_params_)

clf = gs.best_estimator_
clf.fit(X_train, y_train)
print(clf.score(X_train,y_train))
print(clf.score(X_test, y_test))

1.0
{'svc__C': 0.15, 'svc__kernel': 'linear'}
1.0
0.9444444444444444


In [18]:
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid,
                  scoring='accuracy', cv=3)
scores = cross_val_score(gs, X, y, scoring='accuracy', cv=5)

print('CV accuracy: %.8f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.96603175 +/- 0.028


In [19]:
# 층화추출한 것
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold # 분할 내 범주비율이 전체데이터 범주비율과 유사하게
inner_cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=0) # shuffle 해 잘 섞음
outer_cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

gs = GridSearchCV(estimator=pipe_svc,
                  param_grid=param_grid,
                  scoring='accuracy', cv=inner_cv)
scores = cross_val_score(gs, X, y, scoring='accuracy', cv=outer_cv)

print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.966 +/- 0.011


표준화 시키지 않음

In [25]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svc = SVC(random_state=1)

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [{'C': param_range, 'kernel': ['linear']},
              {'C': param_range, 'gamma': param_range,
               'kernel': ['rbf']}]
gs = GridSearchCV(estimator=svc, param_grid=param_grid,
                  scoring='accuracy', cv=10)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

clf = gs.best_estimator_
clf.fit(X_train, y_train)
print(clf.score(X_train,y_train))
print(clf.score(X_test, y_test))

0.9519230769230769
{'C': 0.01, 'kernel': 'linear'}
0.967741935483871
0.9074074074074074


In [18]:
# 에러 뜨면 아래에서 확인하고 초모수값 이름 설정하기
gs.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__C', 'estimator__break_ties', 'estimator__cache_size', 'estimator__class_weight', 'estimator__coef0', 'estimator__decision_function_shape', 'estimator__degree', 'estimator__gamma', 'estimator__kernel', 'estimator__max_iter', 'estimator__probability', 'estimator__random_state', 'estimator__shrinking', 'estimator__tol', 'estimator__verbose', 'estimator', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])

In [26]:
gs = GridSearchCV(estimator=svc, param_grid=param_grid,
                  scoring='accuracy', cv=3)
scores = cross_val_score(gs, X, y, scoring='accuracy', cv=5)

print('CV accuracy: %.8f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.96111111 +/- 0.033


## CART 의사결정 나무

shuffle = T 하고 층화추출함

In [20]:
from sklearn.tree import DecisionTreeClassifier
dc = DecisionTreeClassifier(random_state=0)

param_grid = {'max_depth': [1, 2, 3, 4, 5, 6, 7, None], 'criterion': ['gini', 'entropy']}
gs = GridSearchCV(estimator=dc, param_grid=param_grid,
                  scoring='accuracy', cv=10)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

clf = gs.best_estimator_
clf.fit(X_train, y_train)
print(clf.score(X_train,y_train))
print(clf.score(X_test, y_test))

0.9275641025641026
{'criterion': 'gini', 'max_depth': 4}
1.0
0.9444444444444444


In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold # 분할 내 범주비율이 전체데이터 범주비율과 유사하게
inner_cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=0) # shuffle 해 잘 섞음
outer_cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=0),
                  param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, None]}],
                  scoring='accuracy', cv=inner_cv)
scores = cross_val_score(gs, X, y, scoring='accuracy', cv=outer_cv)

print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.938 +/- 0.041


1. 첫번째 과정에서 초모수 구함

2. test 데이터(즉 traing에 의존하지 않는지 확인하는게 두번째 과정 (첫번째 결과가 너무 크면 운이 좋았던 것 과대적합한거임, 두번째 결과가 더 커야함)


첫번째에서 구한 학습 데이터의 성능과 

두번째에서 구한 시험 데이터의 성능의 평균을 비교해 과대적합의 여부 판단


> 두번째에서 구한 시험 데이터의 성능의 평균이 해당 모형의 성능이 된다.


마지막에 3개 다 비교하기

성능은 accuacy 로 구함 cv 교차검증은 10으로 함
